In [19]:
import pickle
result_root_2A = f"experiment_result/FedAvgM_Diffusion_2A_20250806_005337/intermediate_results"
for i in range(10):
    with open(f"{result_root_2A}/CF_{i}_result.pkl", "rb") as f:
        result = pickle.load(f)
        history_obj = result["history"]
        
        # Access diffusion_loss from metrics_centralized
        print("ssim:", history_obj.metrics_centralized["ssim"][-1][1])

ssim: 0.9166019558906555
ssim: 0.9210359454154968
ssim: 0.3460482954978943
ssim: 0.936425507068634
ssim: 0.895470142364502
ssim: 0.878612756729126
ssim: 0.6202558279037476
ssim: 0.895199179649353
ssim: 0.8271270990371704
ssim: 0.2616521120071411


In [20]:
import pickle
result_root_2A = f"experiment_result/FedAvgM_Diffusion_2A_20250806_005337/intermediate_results"
for i in range(10):
    with open(f"{result_root_2A}/CV_{i}_result.pkl", "rb") as f:
        result = pickle.load(f)
        history_obj = result["history"]
        
        # Access diffusion_loss from metrics_centralized
        print("ssim:", history_obj.metrics_centralized["ssim"][-1][1])

ssim: 0.6860289573669434
ssim: 0.9451403021812439
ssim: 0.9181747436523438
ssim: 0.8302531242370605
ssim: 0.6148689389228821
ssim: 0.8870447874069214
ssim: 0.931169867515564
ssim: 0.4751071631908417
ssim: 0.9097065329551697
ssim: 0.8960686326026917


In [ ]:
import pickle
time_result_2A = f"experiment_result/FedAvgM_Diffusion_2A_20250807_145112/timing_results.pkl"
with open(time_result_2A, "rb") as f:
    time_result = pickle.load(f)
    print(time_result)

{'config_loading': [0.007500410079956055], 'setup_mp_device': [0.011054039001464844], 'fwi_forward_setup': [0.00041937828063964844], 'diffusion_model_setup': [1.2448081970214844], 'ssim_loss_setup': [0.000438690185546875], 'data_loading': [0.05160975456237793], 'initial_model_creation': [0.023332595825195312], 'evaluate_fn_creation': [0.0004584789276123047], 'strategy_creation': [0.00042700767517089844], 'client_fn_creation': [6.4373016357421875e-06], 'fl_simulation': [25.857327699661255], 'result_processing': [0.0003609657287597656], 'intermediate_saving': [0.0002448558807373047], 'family_history_calculation': [0.00015997886657714844], 'overall_history_calculation': [7.486343383789062e-05], 'final_saving': [0.00013875961303710938], 'total_experiment': [27.198987245559692]}
